In [ ]:
!pwd

In [ ]:
!python --version

# 修改checkpoint变量名

In [ ]:
import os
from tensorflow.python import pywrap_tensorflow
model_dir="/root/magenta/magenta/models/rl_tuner/checkpoint/src2/model.ckpt-20000" #checkpoint的文件位置
# Read data from checkpoint file
reader = pywrap_tensorflow.NewCheckpointReader(model_dir)
var_to_shape_map = reader.get_variable_to_shape_map()
# Print tensor name and values
for key in var_to_shape_map:
    print("tensor_name: ", key)  #输出变量名
    #print(reader.get_tensor(key))   #输出变量值

**需要的节点**

Key rnn_model/rnn/multi_rnn_cell/cell_0/lstm_cell/bias

_CHECKPOINTABLE_OBJECT_GRAPH

In [ ]:
!python /root/magenta/magenta/models/rl_tuner/rename_tf_variable.py \
--ckpt_path /root/magenta/magenta/models/rl_tuner/checkpoint/src \
--save_path /root/magenta/magenta/models/rl_tuner/checkpoint/model.ckpt-15321 \
--rename_var_src fully_connected/weights,fully_connected/biases \
--rename_var_dst rnn_model/fully_connected/weights,rnn_model/fully_connected/bias

In [ ]:
import os
from tensorflow.python import pywrap_tensorflow
model_dir='/root/magenta/magenta/models/rl_tuner/checkpoint/dist1/model.ckpt-15321' #checkpoint的文件位置
# Read data from checkpoint file
reader = pywrap_tensorflow.NewCheckpointReader(model_dir)
var_to_shape_map = reader.get_variable_to_shape_map()
# Print tensor name and values
for key in var_to_shape_map:
    print("tensor_name: ", key)  #输出变量名
    #print(reader.get_tensor(key))   #输出变量值

In [1]:
import tensorflow as tf
import numpy as np
import sys
import importlib

In [3]:
from magenta.models.rl_tuner import rl_tuner
from magenta.models.rl_tuner import rl_tuner_ops

In [4]:
#设定输出文件存储路径
SAVE_PATH = "/root/magenta/magenta/models/rl_tuner/output/2020.4.7"

In [5]:
# Model parameter settings
ALGORITHM = 'q'
REWARD_SCALER = 1
OUTPUT_EVERY_NTH = 50000
NUM_NOTES_IN_COMPOSITION = 64
PRIME_WITH_MIDI = False
NOTE_RNN_CHECKPOINT_DIR = '/root/magenta/magenta/models/rl_tuner/checkpoint/dist1/model.ckpt-15321'
NOTE_RNN_TYPE = 'basic_rnn'     #'basic_rnn', 'default'
NOTE_RNN_HPARAMS = "{'batch_size': 64, 'rnn_layer_sizes': [64, 64]}"

In [6]:
rl_tuner_hparams = tf.contrib.training.HParams(random_action_probability=0.1,
                                               store_every_nth=1,
                                               train_every_nth=5,
                                               minibatch_size=32,
                                               discount_rate=0.5,
                                               max_experience=100000,
                                               target_network_update_rate=0.01)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



# Train network

In [7]:
importlib.reload(sys)
importlib.reload(rl_tuner_ops)
importlib.reload(rl_tuner)
rl_tuner.reload_files()

In [8]:
rl_net = rl_tuner.RLTuner(SAVE_PATH, 
                          dqn_hparams=rl_tuner_hparams, 
                          algorithm=ALGORITHM,
                          reward_scaler=REWARD_SCALER,
                          output_every_nth=OUTPUT_EVERY_NTH,
                          num_notes_in_melody=NUM_NOTES_IN_COMPOSITION,
                          note_rnn_checkpoint_dir=NOTE_RNN_CHECKPOINT_DIR,
                          note_rnn_hparams=NOTE_RNN_HPARAMS,
                          note_rnn_type=NOTE_RNN_TYPE)

INFO:tensorflow:Initializing q network
INFO:tensorflow:Using custom hparams
INFO:tensorflow:Initializing melody RNN graph for scope q_network
[64, 64]
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.


TypeError: Error converting shape to a TensorShape: int() argument must be a string, a bytes-like object or a number, not 'tuple'.

In [ ]:
# Generate initial music sequence before training with RL
rl_net.generate_music_sequence(visualize_probs=True, title='pre_rl', length=32)

In [ ]:
rl_net.train(num_steps=1000000, exploration_period=500000)

In [ ]:
# Plot the rewards received during training. Improves as chance of random exploration action decreases.
rl_net.plot_rewards()

In [ ]:
# Plot rewards received during calls to evaluation function throughout training. 
# Does not include exploration or random actions.
rl_net.plot_evaluation()